# **DATA CLEANING PROJECT**

## **SUMARRY**

### **<u>Objectives</u>**

- To make dirty data in to cleaned data
- Remove all needless data
- To make complete data support for analysis process 
- a completely objective table  contains 3 column ( NAICS\_Industry\_Decription , LookupCodes, and Sectors)

### **<u>Dataset</u>**

- Using Table of size standards dataset from [Table of size standards (sba.gov)](https://www.sba.gov/document/support-table-size-standards)
- In **table of size standards sheet** contain 2 column ( NAICS\_Codes and NAICS\_Industry\_Decription)

**<u>Tools</u>**

- SSIS Package (Visual Studio 2019) to import dataset ( CSV file) into database( MS SQL server)
- MS SQL server (Azure data studio) to write code chunks for cleaning data and write documentations

> **First glance at Table of size standards  Dataset**

In [ ]:
SELECT * 
FROM Sba_industry_standard

> **Select all data we need to use for create new table**

In [ ]:
SELECT 
    [NAICS _Codes], 
    [NAICS _Industry _Description]
FROM Sba_industry_standard 
WHERE [NAICS _Codes] =' ' -- this is filter to remove all outliers because every Row in NAICS_Industry_Description column also have blank in NAICS_Codes column


> **Create a new LookupCodes column from NAICS\_Industry\_Description column**

In [ ]:
SELECT 
    [NAICS _Industry _Description],
    SUBSTRING([NAICS _Industry _Description],8,2) 
    -- to extract code numbers from NAICS_Industry_Description column
FROM Sba_industr_standard 
WHERE [NAICS _Codes] =' '

> **Create a new Sectors column from NAICS\_Industry\_Description column**

In [ ]:
SELECT 
    [NAICS _Industry _Description],
    SUBSTRING([NAICS _Industry _Description],8,2) AS LookupCocdes,
    SUBSTRING([NAICS _Industry _Description],CHARINDEX('–',[NAICS _Industry _Description]) +1,LEN(([NAICS _Industry _Description]))) AS Sectors 
    -- to extract string from NAICS_Industry_Description column into a new Sectors Column
FROM Sba_industry_standard 
WHERE [NAICS _Codes] =' '

> **Remove Space before a new string in Sectors column**

In [ ]:
SELECT 
    [NAICS _Industry _Description],
    SUBSTRING([NAICS _Industry _Description],8,2) AS LookupCocdes,
    ltrim(SUBSTRING([NAICS _Industry _Description],CHARINDEX('–',[NAICS _Industry _Description]) +1,LEN(([NAICS _Industry _Description])))) AS Sectors
    --use ltrim() function because space in the left side of string
FROM Sba_industry_standard 
WHERE [NAICS _Codes] =' '

> **Remove all NA values and outliers values in LookupCodes column AND create a new table (sba\_NAICS\_Codes\_Sectors)**

In [ ]:
SELECT*
INTO sba_NAICS_codes_sectors -- Create a new table(sba_NAICS_Codes_sectors) by using Subquery code chunk
FROM 
(

SELECT 
    [NAICS _Industry _Description],
    SUBSTRING([NAICS _Industry _Description],8,2) AS LookupCocdes,
    ltrim(SUBSTRING([NAICS _Industry _Description],CHARINDEX('–',[NAICS _Industry _Description]) +1,LEN(([NAICS _Industry _Description])))) AS Sectors
FROM Sba_industry_standard 
WHERE [NAICS _Codes] =' '

) AS temp_table
WHERE LookupCocdes != 'NA' AND LookupCocdes != 'bu' -- to remove NA values and outlier Values(bu) in LookupCodes column

> **First Glance at a new table (sba\_NAICS\_codes\_sectors) have been created**

In [ ]:
SELECT *
FROM sba_NAICS_codes_sectors


> **We realize some missing codes those are 32, 43, 49 in LookupCodes Column**

> **Insert missing values into a new table (sba\_NAICS\_codes\_sectors)**

In [ ]:
INSERT INTO sba_NAICS_codes_sectors
VALUES  ('Sector 31 – 33 – Manufacturing',32,'Manufacturing'),
        ('Sector 44 - 45 – Retail Trade',43,'Retail Trade'),
        ('Sector 48 - 49 – Transportation and Warehousing',49,'Transportation and Warehousing')

> **We realize in a sectors column have an error string ( "33-Manufacturing")**

> **Fix  an error string ( "33-Manufacturing") by using UPDATE function**

In [ ]:
UPDATE sba_NAICS_codes_sectors
SET Sectors ='Manufacturing'
WHERE [NAICS _Industry _Description] = 'Sector 31 – 33 – Manufacturing'

> **Select all data from a completely table have just been created**

>

In [ ]:
SELECT *
FROM sba_NAICS_codes_sectors
ORDER BY LookupCocdes
-- This is a completely objective table that we need to create, it's contain 3 column (NAICS_Industry_Description, LookupCodes, Sectors)